In [1]:
from urllib import request
from urllib.request import Request
from bs4 import BeautifulSoup
from lxml import etree
import requests
import pandas as pd
import json
import time
from pandas.io.json import json_normalize
import re

In [2]:
# 根据Json文件的请求链接得到主菜单的排序方式，于是制作链接片段
url_forwards = 'https://www.tmall.com/wow/chaoshi/act/catpopup?wh_id='
url_back = '&wh_logica=HD&wh_callback=__chaoshi_category_popup_'
url_small = ['jksp','spyl','lyfs','mrxh','jjjd','jtqj','myyp']

In [3]:
# 制作链接列表
result = []
for i in url_small:
    result.append(url_forwards+i+url_back+i)
# 遍历列表，访问服务器，得到所有的Json结果
JsonResult = []
for j in result:
    JsonResult.append(requests.get(j))
    print(j)
    time.sleep(5)

https://www.tmall.com/wow/chaoshi/act/catpopup?wh_id=jksp&wh_logica=HD&wh_callback=__chaoshi_category_popup_jksp
https://www.tmall.com/wow/chaoshi/act/catpopup?wh_id=spyl&wh_logica=HD&wh_callback=__chaoshi_category_popup_spyl
https://www.tmall.com/wow/chaoshi/act/catpopup?wh_id=lyfs&wh_logica=HD&wh_callback=__chaoshi_category_popup_lyfs
https://www.tmall.com/wow/chaoshi/act/catpopup?wh_id=mrxh&wh_logica=HD&wh_callback=__chaoshi_category_popup_mrxh
https://www.tmall.com/wow/chaoshi/act/catpopup?wh_id=jjjd&wh_logica=HD&wh_callback=__chaoshi_category_popup_jjjd
https://www.tmall.com/wow/chaoshi/act/catpopup?wh_id=jtqj&wh_logica=HD&wh_callback=__chaoshi_category_popup_jtqj
https://www.tmall.com/wow/chaoshi/act/catpopup?wh_id=myyp&wh_logica=HD&wh_callback=__chaoshi_category_popup_myyp


In [15]:
JsonToloads = []
# Json的原始数据不规范，先对原始数据进行处理，取(*)之间的值
for i in JsonResult:
    JsonToloads.append(re.split(r'\(|\)',i.text)[1])

In [16]:
# 将Json数据转为DataFrame数据
def JsonToDataFrame(result):
    mergeOne = json_normalize(result['data'],'cats')[['title','link']]

    mergeTwo = json_normalize(result['data']['cats'],'cats',['cats','title'])[['title','text','link']]
    mergeTwo.rename(columns={'link':'productLink'},inplace=True)
    result_save = pd.merge(mergeOne,mergeTwo,on = 'title')

    acts = json_normalize(result['data']['acts'])
    # 这个判断是为了判断出活动是否为空，为空不执行这一步
    if acts.shape[0]!=0:
        acts = acts[['text','link']]
        acts.rename(columns={'link':'productLink'},inplace=True)
        result_save = pd.merge(result_save,acts,how = 'outer')
    # 这个判断是为了判断出品牌是否为空，为空不执行
    brands = json_normalize(result['data']['brands'])
    if brands.shape[0]!=0:
        brands = brands[['text','link']]
        brands.rename(columns={'link':'productLink'},inplace=True)
        result_save = pd.merge(result_save,brands,how = 'outer')

    return result_save

In [17]:
# 创建list作为一级菜单
url_small_name = ['进口食品','食品饮料','粮油副食','美容洗护','家居家电','家庭清洁','母婴用品']

for i in range(len(JsonToloads)):
    name = url_small[i]
    values = json.loads(JsonToloads[i])
    if i == 0:
        result = JsonToDataFrame(values)
        result['Main']=url_small_name[i]
    else:
        temp = JsonToDataFrame(values)
        # 增加一级菜单列
        temp['Main']=url_small_name[i]
        result = pd.merge(result,temp,how='outer')
    print(i,end='0%-')

00%-10%-20%-30%-40%-50%-60%-

In [18]:
result['link'] = result['link'].str.replace('//list.','https://list.')
result['productLink'] = result['productLink'].str.replace('//list.','https://list.')

In [19]:
result

,title,link,text,productLink,Main
0,进口牛奶,https://list.tmall.com/search_product.htm?spm=...,进口纯牛奶,https://list.tmall.com/search_product.htm?spm=...,进口食品
1,进口牛奶,https://list.tmall.com/search_product.htm?spm=...,进口酸奶,https://list.tmall.com/search_product.htm?spm=...,进口食品
2,进口牛奶,https://list.tmall.com/search_product.htm?spm=...,进口成人奶粉,https://list.tmall.com/search_product.htm?spm=...,进口食品
3,进口牛奶,https://list.tmall.com/search_product.htm?spm=...,进口含乳饮料,https://list.tmall.com/search_product.htm?spm=...,进口食品
4,进口牛奶,https://list.tmall.com/search_product.htm?spm=...,进口有机牛奶,https://list.tmall.com/search_product.htm?spm=...,进口食品
5,进口牛奶,https://list.tmall.com/search_product.htm?spm=...,进口酸奶粉,https://list.tmall.com/search_product.htm?spm=...,进口食品
6,进口零食,https://list.tmall.com/search_product.htm?spm=...,进口蜜饯果干,https://list.tmall.com/search_product.htm?spm=...,进口食品
7,进口零食,https://list.tmall.com/search_product.htm?spm=...,进口薯片/膨化,https://list.tmall.com/search_product.htm?spm=...,进口食品
8,进口零食,https://list.tmall.com/search_product.htm?spm=...,进口坚果,https://list.tmall.com/search_product.htm?spm=...,进口食品
9,进口零食,https://list.tmall.com/search_product.htm?spm=...,进口糖果,https://list.tmall.com/search_product.htm?spm=...,进口食品


In [20]:
# Save data
result[['Main','title','link','text','productLink']].to_csv('Items.csv')

In [4]:
import pandas as pd

In [5]:
a = [1,1,2,3,5,]

In [7]:
a = pd.DataFrame(a)

In [ ]:
a.to_